In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

In [23]:
dataset = pd.read_csv("Data.csv")
dataset.head(5)

,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,roloff_std,...,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo,genre
0,4031.364538,1027.079258,1784.420446,360.263361,5.246790,1.537974,7644.287109,1830.322266,3806.485316,949.558806,...,-0.189109,-0.110339,22.482096,-26.569305,-3.667368,6.422199,0.313302,-0.037197,123.046875,blues
1,5444.293577,421.091729,1529.835316,613.175361,4.028086,1.339397,8085.717773,333.764648,3548.820207,1725.249654,...,0.054884,0.154526,19.230034,-27.189579,-2.239120,8.055376,-0.182963,0.004210,107.666016,blues
2,4669.770713,417.254802,1552.481958,395.634070,11.609763,1.930126,6556.860352,441.430664,3040.514948,884.938380,...,0.542265,-0.682770,23.078028,-28.854303,-8.905224,7.774160,0.489465,0.710559,161.499023,blues
3,3236.442116,318.713500,1070.119953,429.532791,1.803354,1.001449,7611.987305,387.597656,2185.028454,1222.395194,...,1.569782,0.613722,20.651333,-32.890564,-2.476421,7.538023,0.499351,-0.163374,172.265625,blues
4,4464.156642,849.658285,1835.494603,585.961468,1.536058,1.049734,7084.423828,1152.026367,3580.945013,1253.915665,...,0.430401,0.456069,24.070387,-36.607307,-6.934123,9.466887,0.057030,0.132279,135.999178,blues


In [40]:
X = dataset.drop(['genre'], axis=1)
y = dataset['genre']

scale = StandardScaler()
X_scaled = pd.DataFrame(scale.fit_transform(X), columns = X.columns)

encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_enc, test_size=0.3, random_state=42)

In [41]:
X_train.shape

(700, 139)

In [42]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
700/700 [==============================] - 0s 353us/step - loss: 2.1725 - accuracy: 0.2386
Epoch 2/20
700/700 [==============================] - 0s 23us/step - loss: 1.6123 - accuracy: 0.4371
Epoch 3/20
700/700 [==============================] - 0s 23us/step - loss: 1.2526 - accuracy: 0.5929
Epoch 4/20
700/700 [==============================] - 0s 23us/step - loss: 0.9861 - accuracy: 0.7343
Epoch 5/20
700/700 [==============================] - 0s 21us/step - loss: 0.7863 - accuracy: 0.7700
Epoch 6/20
700/700 [==============================] - 0s 23us/step - loss: 0.6195 - accuracy: 0.8271
Epoch 7/20
700/700 [==============================] - 0s 21us/step - loss: 0.4978 - accuracy: 0.8743
Epoch 8/20
700/700 [==============================] - 0s 21us/step - loss: 0.4130 - accuracy: 0.8943
Epoch 9/20
700/700 [==============================] - 0s 23us/step - loss: 0.3317 - accuracy: 0.9143
Epoch 10/20
700/700 [==============================] - 0s 23us/step - loss: 0.2671 - accur

In [43]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

300/300 [==============================] - 0s 149us/step
test_acc:  0.7433333396911621


In [44]:
partial_x_train, x_val, partial_y_train, y_val = train_test_split(X_scaled, y_enc, test_size=0.1, random_state=42)

model2 = Sequential()
model2.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))

Train on 900 samples, validate on 100 samples
Epoch 1/30
900/900 [==============================] - 0s 401us/step - loss: 2.2975 - accuracy: 0.1333 - val_loss: 2.1350 - val_accuracy: 0.2300
Epoch 2/30
900/900 [==============================] - 0s 21us/step - loss: 1.9915 - accuracy: 0.3556 - val_loss: 1.8779 - val_accuracy: 0.3600
Epoch 3/30
900/900 [==============================] - 0s 21us/step - loss: 1.7160 - accuracy: 0.4478 - val_loss: 1.6463 - val_accuracy: 0.4300
Epoch 4/30
900/900 [==============================] - 0s 21us/step - loss: 1.4753 - accuracy: 0.4700 - val_loss: 1.4565 - val_accuracy: 0.4500
Epoch 5/30
900/900 [==============================] - 0s 21us/step - loss: 1.2528 - accuracy: 0.5667 - val_loss: 1.2570 - val_accuracy: 0.5400
Epoch 6/30
900/900 [==============================] - 0s 20us/step - loss: 1.0654 - accuracy: 0.6622 - val_loss: 1.0998 - val_accuracy: 0.6100
Epoch 7/30
900/900 [==============================] - 0s 25us/step - loss: 0.9067 - accuracy: 0

In [45]:
test_loss, test_acc = model2.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

300/300 [==============================] - 0s 40us/step
test_acc:  0.9233333468437195
